In [3]:
from gym.core import Env
from autolab.autolab import Potentiostat
from secm.secm import SECM

In [ ]:
from typing import Optional, Tuple, Union


class OerEnvironment (Env):

    def __init__(self, potentiostat: Potentiostat, secm: SECM) -> None:
        
        #TODO: how to check if potentiostat is connected
        self.potentiostat = potentiostat
        self.secm = secm
        self.step_size = 2500
        
        self.action_space = None
        self.observation_space = None
        self.reward_range = None
        self.spec = None


    def step(self, action: Any) -> Tuple[Any, float, bool, dict]:
        return super().step(action)
    
    def reset(self) -> None:
        self.secm.move_to_next_experiment(self.step_size)
    
    def close(self):
        self.secm.move_to_wash()
        #TODO: shut off and disconnect potentiostat